In [1]:
import numpy as np
import librosa as lr
import pickle

import os

In [9]:
TRAIN_DIR = "../../raw-dataset/data_splits/train"
DEV_DIR = "../../raw-dataset/data_splits/dev"

# Loading Train and Dev Data

In [10]:
train_files = sorted(os.listdir(TRAIN_DIR))
dev_files = sorted(os.listdir(DEV_DIR)) 

In [4]:
# create spectrogram

def create_spectrogram(file):
    y, sr = lr.load(file)
    S = lr.feature.melspectrogram(y=y, sr=sr)
    S = lr.power_to_db(S, ref=np.max)
    # convert the dtype to float32 for reducing the size
    S = S.astype(np.float32)
    return S

In [5]:
all_spectrograms = []

for file in train_files:
    file_id = file.split("_")[0]
    spectogram = create_spectrogram(os.path.join(TRAIN_DIR, file, f"{file_id}_AUDIO.wav"))
    all_spectrograms.append(spectogram)

In [6]:
# save spectrograms
with open("train_spectograms.pkl", "wb") as f:
    pickle.dump(all_spectrograms, f)

In [8]:
del (all_spectrograms, train_files)

In [13]:
# create dev data spectograms
dev_spectograms = []

for file in dev_files:
    file_id = file.split("_")[0]
    spectogram = create_spectrogram(os.path.join(DEV_DIR, file, f"{file_id}_AUDIO.wav"))
    dev_spectograms.append(spectogram)

In [19]:
# save spectrograms
with open("dev_spectograms.pkl", "wb") as f:
    pickle.dump(dev_spectograms, f)

# Padding Spectograms

In [2]:
def pad_spectogram(all_spectograms):
    MAX_SHAPE = max([spect.shape[1] for spect in all_spectograms])
    # print(MAX_SHAPE)
    
    padded_spectograms = np.zeros((len(all_spectograms), all_spectograms[0].shape[0], MAX_SHAPE), dtype=np.float16)
    # print(padded_spectograms.shape)
    
    for i in range(len(all_spectograms)):
        spectogram = all_spectograms[i]
        (row, col) = spectogram.shape
        pad_len = MAX_SHAPE - col
        padded_spect = np.pad(spectogram, ((0,0),(0, pad_len)), mode="constant")
        
        padded_spectograms[i] = padded_spect
        
    return padded_spectograms

In [3]:
with open("train_spectograms.pkl", "rb") as f:
    train_spectograms = pickle.load(f)

In [4]:
padded_train_spectograms = pad_spectogram(train_spectograms)

In [5]:
np.save("padded_train_spectograms.npy", padded_train_spectograms)